<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/%EB%B2%84%EC%8A%A4_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 버스 운행시간 데이터를 활용한 데이터 분석
- 버스가 다음 정류장에 도착하기 위해서 걸릴 시간을 예측

## 1. 라이브러리 및 데이터
## Library & Data

In [2]:
## matplotlib 사용시 한국어 나오게 하는 코드

!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

# fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') 



In [3]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb #모델링 lgb
import folium #python에서 지도를 다루는 라이브러리

In [5]:
# 데이터 불러오기
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
submission=pd.read_csv('/content/submission_제출양식.csv')

## 2. 탐색적 자료분석
## Exploratory Data Analysis (EDA)

### pd.DataFrame.head()
 - 데이터 프레임의 위에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [6]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24.0
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36.0
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40.0
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42.0
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64.0


+ id : 고유 id
+ date : 버스 운행 날짜
+ route_id : 버스 노선 ID
+ vh_id : 버스 id
+ route_nm : 버스 노선 실제 번호
+ now_latitude : 현재 정류소의 위도
+ now_longitude : 현재 정류소의 경도
+ now_station : 현재 정류소 이름
+ now_arrive_time : 현재 정류장에 도착한 시간
+ distance : 현재 정류장에서 다음 정류장까지 실제 이동한 거리
+ next_station : 다음 정류소 이름
+ next_latitude : 다음 정류소의 위도
+ next_longitude : 다음 정류소의 경도
+ next_arrive_time : 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값


In [7]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001.0,7997025.0,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001.0,7997025.0,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001.0,7997025.0,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001.0,7997025.0,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001.0,7997025.0,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [8]:
print(train['date'].min())
print(train['date'].max())
print(test['date'].min())
print(test['date'].max())

2019-10-15
2019-10-18
2019-10-29
2019-11-03


### pd.DataFrame.tail()
 - 데이터 프레임의 아래에서 부터 n개 행을 보여주는 함수
 - n의 기본 값(default 값)은 5

In [9]:
train.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
56597,56597,2019-10-18,405320116,7983036,201-16,33.519583,126.570783,제주동중학교,20시,542.0,화북남문,33.518900,126.565117,96.0
56598,56598,2019-10-18,405320116,7983036,201-16,33.518900,126.565117,화북남문,20시,350.0,화북초등학교,33.517617,126.561717,34.0
56599,56599,2019-10-18,405320116,7983036,201-16,33.517617,126.561717,화북초등학교,20시,508.0,오현중고등학교,33.515183,126.557067,74.0
56600,56600,2019-10-18,405320116,7983036,201-16,33.515183,126.557067,오현중고등학교,20시,97.0,거로 입구,33.514637,126.556243,14.0
56601,56601,2019-10-18,405320116,7983036,201-16,33.512475,126.549973,국립제주박물관,20시,208.0,우당도서관,33.000000,NaN,NaN


### pd.DataFrame.shape
 - 데이터 프레임의 행의 개수와 열의 개수가 저장되어 있는 속성(attribute)

In [10]:
print(train.shape)
print(test.shape)
print(submission.shape)

(56602, 14)
(59803, 13)
(91774, 2)


### pd.DataFrame.info()
- 데이터셋의 column별 정보를 알려주는 함수
- 비어 있지 않은 값은 (non-null)은 몇개인지?
- column의 type은 무엇인지?
 - type의 종류 : int(정수), float(실수), object(문자열), 등등 (date, ...)

In [11]:
train.info() # 결측치 없음!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56602 entries, 0 to 56601
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                56602 non-null  int64  
 1   date              56602 non-null  object 
 2   route_id          56602 non-null  int64  
 3   vh_id             56602 non-null  int64  
 4   route_nm          56602 non-null  object 
 5   now_latitude      56602 non-null  float64
 6   now_longitude     56602 non-null  float64
 7   now_station       56602 non-null  object 
 8   now_arrive_time   56602 non-null  object 
 9   distance          56602 non-null  float64
 10  next_station      56602 non-null  object 
 11  next_latitude     56602 non-null  float64
 12  next_longitude    56601 non-null  float64
 13  next_arrive_time  56601 non-null  float64
dtypes: float64(6), int64(3), object(5)
memory usage: 6.0+ MB


### pd.DataFrame.describe()
- 숫자형 (int, float) column들의 기술 통계량을 보여주는 함수

- 기술통계량이란?
 - 해당 column을 대표할 수 있는 통계값들을 의미
 
 
- 기술통계량 종류
 - count: 해당 column에서 비어 있지 않은 값의 개수
 - mean: 평균
 - std: 표준편차
 - min: 최솟값 (이상치 포함)
 - 25% (Q1): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 1/4번째 지점에 있는 값
 - 50% (Q2): 중앙값 (전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 2/4번째 지점에 있는 값)
 - 75% (Q3): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 3/4번째 지점에 있는 값
 - max: 최댓값 (이상치 포함) 
 
 
 
- 이상치: 울타리 밖에 있는 부분을 이상치라고 정의함
   - 아래쪽 울타리: $Q_1$ - $1.5 * IQR$
   - 위쪽 울타리: $Q_3$ + $1.5 * IQR$
   - $IQR$ = $Q_3 - Q_1$
 
 
<img src="https://miro.medium.com/max/10125/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width="700" height="500">

In [12]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,56602.000000,5.660200e+04,5.660200e+04,56602.000000,56602.000000,56602.000000,56602.000000,56601.000000,56601.000000
mean,28300.500000,4.052388e+08,7.989443e+06,33.439887,126.598518,482.546942,33.440028,126.598809,87.186728
std,16339.734305,9.278169e+04,6.900365e+03,0.098820,0.123408,487.540292,0.098721,0.123272,88.644907
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.000000,126.473300,6.000000
25%,14150.250000,4.051365e+08,7.983093e+06,33.386021,126.520964,292.000000,33.391244,126.521267,44.000000
50%,28300.500000,4.053201e+08,7.983445e+06,33.485199,126.545689,383.000000,33.485199,126.545821,68.000000
75%,42450.750000,4.053201e+08,7.997062e+06,33.500197,126.638767,541.000000,33.500228,126.638767,104.000000
max,56601.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


### pd.DataFrame.groupby()
 - 집단에 대한 통계량 확인 
 
<img src="https://s3.amazonaws.com/files.dezyre.com/images/Tutorials/Split+Apply+Combine+Strategy.png" width="700" height="500">

### folium.Map()
 - location 옵션에 있는 좌표를 중심으로 지도 시각화

In [13]:
#부산시 좌표
coord = [35.1559361, 129.1384361] 
map_osm = folium.Map(location = coord)
map_osm

In [14]:
map_osm = folium.Map(location=coord, zoom_start=12) #zoom_start = 확대
map_osm

In [15]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain') # 지도 시각화 종류류
map_osm

In [16]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner')
map_osm

In [17]:
# 마커 찍기 Marker
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location=coord, zoom_start=12)

folium.Marker(coord, popup='부산대학교', tooltip = '부산대학교 tooltip').add_to(map_osm)
 # 마커 생성 뒤 지도에 추가해주는 기능

map_osm.save('index.html')

map_osm

default images: https://getbootstrap.com/docs/3.3/components/

- 검색 기능이 없다

fontawesome:https://fontawesome.com/icons?d=gallery

- 해당링크의 무료인것만 쓸 수 있음

In [18]:
map_osm = folium.Map(location=coord, zoom_start=12)
folium.Marker(coord, popup='부산대학교', icon=folium.Icon(icon='star')).add_to(map_osm)
# icon으로 마커 모양 바꿀 수 있음
map_osm

In [19]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.Marker([37.566345, 126.977893], popup='서울특별시청', icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
folium.Circle([37.4600, 126.9519], tooltip='서울대학교', radius=3000, color="green").add_to(map_osm)
folium.CircleMarker([37.5662, 126.9386], radius=10, popup='연세대학교',color="black", fill_color="crimson").add_to(map_osm)
map_osm

In [20]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.RegularPolygonMarker([37.566345, 126.977893], popup='서울특별시청', fill_color='red',color='red', number_of_sides=3, radius=30, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.4600, 126.9519], popup='서울대학교', fill_color='green',color='green', number_of_sides=4, radius=10, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.5662, 126.9386], popup='연세대학교',fill_color='blue',color='blue', number_of_sides=6, radius=10).add_to(map_osm)
map_osm

In [21]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405320122].drop_duplicates("now_station")


map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)


for item in map_bus_route.index:
    lat = map_bus_route.loc[item,'now_latitude']
    long = map_bus_route.loc[item,'now_longitude']
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm)

map_osm.save('index.html')

map_osm

## 3. 데이터 전처리
## Data Cleansing & Pre-Processing  

### pd.Series.str.slice()
- Series 내에 문자열을 일괄적으로 slicing (자르기)

In [22]:
train['now_arrive_time']=train['now_arrive_time'].str.slice(stop=-1).astype('int')

In [23]:
test['now_arrive_time']=test['now_arrive_time'].str.slice(stop=-1).astype('int')

ValueError: ignored

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [24]:
features=['now_latitude','now_longitude','now_arrive_time','distance']
target=['next_arrive_time']

In [25]:
X_train, X_test, y_train=train[features],test[features],train[target]

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [27]:
model_dict={
    'linear':LinearRegression(),
    'rf':RandomForestRegressor(random_state=0,n_jobs=-1),
    'lgbm':lgb.LGBMRegressor(random_state=0)
}

In [28]:
model_dict

{'linear': LinearRegression(),
 'rf': RandomForestRegressor(n_jobs=-1, random_state=0),
 'lgbm': LGBMRegressor(random_state=0)}

In [29]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [30]:
model_dict['linear']

LinearRegression()

In [ ]:
model_result={}

for key in model_dict.keys():

  print('### 훈련 중 ###')
  model_dict[key].fit(X_train,y_train)

  print('### 예측 중 ###')
  model_result[key] = model_dict[key].predict(X_test)
     

In [32]:
lr_submit = submission.copy()
rf_submit = submission.copy()
lgbm_submit = submission.copy()

In [ ]:
lr_submit['next_arrive_time']=model_result['linear']
rf_submit['next_arrive_time']=model_result['rf']
lgbm_submit['next_arrive_time']=model_result['lgbm']

In [ ]:
lr_submit.to_csv['lr_submit.csv',index=False)
lr_submit.to_csv['rf_submit.csv',index=False)
lr_submit.to_csv['lgbm_submit.csv',index=False)